In [50]:
import pandas as pd
import geopandas as gpd

In [51]:
watersheds = gpd.read_file('../data/8_dig_huc/wbdhu8_a_tn.shp')
print(watersheds.crs)
watersheds.head()

epsg:32136


,REGION,SUBREGION,BASIN,SUBBASIN,HUC_2,HUC_4,HUC_6,HUC_8,ACRES,SQ_MILES,HU_8_STATE,FIPS_C,geometry
0,Lower Mississippi Region,Lower Mississippi-Hatchie,Hatchie-Obion,Bayou De Chien-Mayfield,08,0801,080102,08010201,621099.0,970.0,KY TN,47131 47183 21007 21035 21039 21075 21083 2110...,"POLYGON ((349591.116 300318.427, 349619.051 30..."
1,Lower Mississippi Region,Lower Mississippi-Hatchie,Hatchie-Obion,Obion,08,0801,080102,08010202,841854.0,1315.0,KY TN,47045 47053 47079 47095 47097 47131 47183 2103...,"POLYGON ((375558.716 254946.095, 375574.360 25..."
2,Lower Mississippi Region,Lower Mississippi-Hatchie,Hatchie-Obion,South Fork Obion,08,0801,080102,08010203,740948.0,1158.0,TN,47017 47053 47077 47079 47131 47183,"POLYGON ((345917.827 229854.985, 345977.745 22..."
3,Lower Mississippi Region,Lower Mississippi-Hatchie,Hatchie-Obion,North Fork Forked Deer,08,0801,080102,08010204,610687.0,954.0,TN,47017 47033 47045 47053 47077 47113,"POLYGON ((295445.238 206787.434, 295428.558 20..."
4,Lower Mississippi Region,Lower Mississippi-Hatchie,Hatchie-Obion,South Fork Forked Deer,08,0801,080102,08010205,679073.0,1061.0,TN,47023 47033 47045 47075 47077 47097 47109 47113,"POLYGON ((292003.198 191223.789, 292076.015 19..."


In [52]:
spec = pd.read_csv('../data/rare_species_by_watershed.csv')
spec.head()

,Huc 8,Huc 8 Name,Huc 12,Huc 12 Name,Type,Category,Scientific Name,Common Name,Global Rank,State Rank,Fed Status,State Status,Habitat Description,Wet Habitat Flag
0,3150101,Conasauga River,31501010101,Headwaters Conasuaga River,Vertebrate Animal,Fish,Notropis asperifrons,Burrhead Shiner,G4,S2,--,"Rare, Not State Listed",Rocky and sandy pools and runs of clear creeks...,Aquatic
1,3150101,Conasauga River,31501010101,Headwaters Conasuaga River,Vertebrate Animal,Fish,Notropis stilbius,Silverstripe Shiner,G4,S3,--,"Rare, Not State Listed",Still or gently flowing pool areas in the Cona...,Aquatic
2,3150101,Conasauga River,31501010101,Headwaters Conasuaga River,Invertebrate Animal,Mollusc,Elliptio arctata,Delicate Spike,G2G3Q,S2,--,"Rare, Not State Listed","In substrates of coarse sand, gravel, and cobb...",Aquatic
3,3150101,Conasauga River,31501010101,Headwaters Conasuaga River,Vertebrate Animal,Fish,Notropis chrosomus,Rainbow Shiner,G4,S3,--,D,"Small, clear, often spring-fed streams with sa...",Aquatic
4,3150101,Conasauga River,31501010101,Headwaters Conasuaga River,Invertebrate Animal,Mollusc,Villosa vibex,Southern Rainbow,G5,S2,--,"Rare, Not State Listed",Mud or soft sand in small rivers & creeks in a...,Aquatic


In [53]:
spec = spec.drop(columns = ['Huc 12', 'Huc 12 Name'])

In [54]:
spec.columns = ['Huc_8', 'Watershed', 'Type', 'Category', 'Scientific_Name', 'Common_Name', 
               'Global_Rank', 'State_Rank', 'Fed_Status', 'State_Status', 'Habitat', 'Wet_Habitat_Flag']

In [55]:
spec['summer'] = 1
spec['zero'] = 0

In [56]:
spec['Huc_8'] = spec['zero'].astype(str) + spec['Huc_8'].astype(str)

Create 1's for global rank columns:

In [57]:
spec['G5BOOL'] = spec['Global_Rank'].isin(['G5'])
spec['G4BOOL'] = spec['Global_Rank'].isin(['G4'])
spec['G3BOOL'] = spec['Global_Rank'].isin(['G3'])
spec['G2BOOL'] = spec['Global_Rank'].isin(['G2'])
spec['G1BOOL'] = spec['Global_Rank'].isin(['G1'])
spec['G12BOOL'] = spec['Global_Rank'].isin(['G1G2'])
spec['G23BOOL'] = spec['Global_Rank'].isin(['G2G3'])
spec['G34BOOL'] = spec['Global_Rank'].isin(['G3G4'])
spec['G45BOOL'] = spec['Global_Rank'].isin(['G4G5'])

In [59]:
spec['G5'] = spec['G5BOOL'].astype(int)
spec['G4'] = spec['G4BOOL'].astype(int)
spec['G3'] = spec['G3BOOL'].astype(int)
spec['G2'] = spec['G2BOOL'].astype(int)
spec['G1'] = spec['G1BOOL'].astype(int)
spec['G12'] = spec['G12BOOL'].astype(int)
spec['G23'] = spec['G23BOOL'].astype(int)
spec['G34'] = spec['G34BOOL'].astype(int)
spec['G45'] = spec['G45BOOL'].astype(int)

In [60]:
spec = spec.drop(columns = ['G5BOOL', 'G4BOOL', 'G3BOOL', 'G2BOOL', 'G1BOOL', 'G12BOOL', 'G23BOOL', 
                           'G34BOOL', 'G45BOOL'])

Now do the same for State ranks:

In [62]:
spec['S5BOOL'] = spec['State_Rank'].isin(['S5'])
spec['S4BOOL'] = spec['State_Rank'].isin(['S4'])
spec['S3BOOL'] = spec['State_Rank'].isin(['S3'])
spec['S2BOOL'] = spec['State_Rank'].isin(['S2'])
spec['S1BOOL'] = spec['State_Rank'].isin(['S1'])
spec['S12BOOL'] = spec['Global_Rank'].isin(['S1S2'])
spec['S23BOOL'] = spec['Global_Rank'].isin(['S2S3'])
spec['S34BOOL'] = spec['Global_Rank'].isin(['S3S4'])

In [63]:
spec['S5'] = spec['S5BOOL'].astype(int)
spec['S4'] = spec['S4BOOL'].astype(int)
spec['S3'] = spec['S3BOOL'].astype(int)
spec['S2'] = spec['S2BOOL'].astype(int)
spec['S1'] = spec['S1BOOL'].astype(int)
spec['S12'] = spec['S12BOOL'].astype(int)
spec['S23'] = spec['S23BOOL'].astype(int)
spec['S34'] = spec['S34BOOL'].astype(int)

In [64]:
spec = spec.drop(columns = ['S5BOOL', 'S4BOOL', 'S3BOOL', 'S2BOOL', 'S1BOOL', 'S12BOOL', 'S23BOOL', 'S34BOOL'])

Now do the same for Federal Status:

In [65]:
spec['LEBOOL'] = spec['Fed_Status'].isin(['LE'])
spec['LTBOOL'] = spec['Fed_Status'].isin(['LT'])

In [66]:
spec['LE'] = spec['LEBOOL'].astype(int)
spec['LT'] = spec['LTBOOL'].astype(int)

In [67]:
spec = spec.drop(columns = ['LEBOOL', 'LTBOOL'])

Now do this for Type:

In [68]:
spec['fungusBOOL'] = spec['Type'].isin(['Fungus'])
spec['ITESCBOOL'] = spec['Type'].isin(['International Terrestrial Ecological System Classification'])
spec['An_AssBOOL'] = spec['Type'].isin(['Animal Assemblage'])
spec['IVCNBOOL'] = spec['Type'].isin(['International Vegetation Classification - Natural'])
spec['NV_PlantBOOL'] = spec['Type'].isin(['Nonvascular Plant'])
spec['IN_AnimBOOL'] = spec['Type'].isin(['Invertebrate Animal'])
spec['VB_AnimBOOL'] = spec['Type'].isin(['Vertebrate Animal'])
spec['V_PlantBOOL'] = spec['Type'].isin(['Vascular Plant'])

In [69]:
spec['T_Fungus'] = spec['fungusBOOL'].astype(int)
spec['T_IntnlTer_EcoSystem'] = spec['ITESCBOOL'].astype(int)
spec['T_An_Asslg'] = spec['An_AssBOOL'].astype(int)
spec['T_IntnlVegClass'] = spec['IVCNBOOL'].astype(int)
spec['T_NV_Plant'] = spec['NV_PlantBOOL'].astype(int)
spec['T_INV_Animal'] = spec['IN_AnimBOOL'].astype(int)
spec['T_VBRT_Animal'] = spec['VB_AnimBOOL'].astype(int)
spec['T_VASC_Plant'] = spec['V_PlantBOOL'].astype(int)

In [70]:
spec = spec.drop(columns = ['fungusBOOL', 'ITESCBOOL', 'An_AssBOOL', 'IVCNBOOL', 'NV_PlantBOOL', 
                           'IN_AnimBOOL', 'VB_AnimBOOL', 'V_PlantBOOL'])

Now do this for Category:

In [71]:
spec['PlanarianBOOL'] = spec['Category'].isin(['Planarian'])
spec['TardigradeBOOL'] = spec['Category'].isin(['Tardigrade'])
spec['AnnelidBOOL'] = spec['Category'].isin(['Annelid'])
spec['LiverwortBOOL'] = spec['Category'].isin(['Liverwort'])
spec['Pl_GymnoBOOL'] = spec['Category'].isin(['Plant: Gymnosperm'])
spec['OtherBOOL'] = spec['Category'].isin(['Other Type'])
spec['NDBOOL'] = spec['Category'].isin(['No Data'])
spec['ArachnidBOOL'] = spec['Category'].isin(['Arachnid'])
spec['Pl_CommBOOL'] = spec['Category'].isin(['Plant Community'])
spec['FernBOOL'] = spec['Category'].isin(['Fern and Fern Ally'])
spec['ReptileBOOL'] = spec['Category'].isin(['Reptile'])
spec['CrustaceanBOOL'] = spec['Category'].isin(['Crustacean'])
spec['NV_PLBOOL'] = spec['Category'].isin(['Non-Vascular Plant'])
spec['AmphibianBOOL'] = spec['Category'].isin(['Amphibian'])
spec['BirdBOOL'] = spec['Category'].isin(['Bird'])
spec['InsectBOOL'] = spec['Category'].isin(['Insect'])
spec['MammalBOOL'] = spec['Category'].isin(['Mammal'])
spec['FishBOOL'] = spec['Category'].isin(['Fish'])
spec['MolluscBOOL'] = spec['Category'].isin(['Mollusc'])
spec['Fl_PlBOOL'] = spec['Category'].isin(['Flowering Plant'])

In [72]:
spec['C_Planarian'] = spec['PlanarianBOOL'].astype(int)
spec['C_Tardigrade'] = spec['TardigradeBOOL'].astype(int)
spec['C_Annelid'] = spec['AnnelidBOOL'].astype(int)
spec['C_Liverwort'] = spec['LiverwortBOOL'].astype(int)
spec['C_Pl_Gymnosperm'] = spec['Pl_GymnoBOOL'].astype(int)
spec['C_Other'] = spec['OtherBOOL'].astype(int)
spec['C_No_Data'] = spec['NDBOOL'].astype(int)
spec['C_Arachnid'] = spec['ArachnidBOOL'].astype(int)
spec['C_Plt_Community'] = spec['Pl_CommBOOL'].astype(int)
spec['C_Fern'] = spec['FernBOOL'].astype(int)
spec['C_Reptile'] = spec['ReptileBOOL'].astype(int)
spec['C_Crustacean'] = spec['CrustaceanBOOL'].astype(int)
spec['C_Non_Vasc_Plant'] = spec['NV_PLBOOL'].astype(int)
spec['C_Amphibian'] = spec['AmphibianBOOL'].astype(int)
spec['C_Bird'] = spec['BirdBOOL'].astype(int)
spec['C_Insect'] = spec['InsectBOOL'].astype(int)
spec['C_Mammal'] = spec['MammalBOOL'].astype(int)
spec['C_Fish'] = spec['FishBOOL'].astype(int)
spec['C_Mollusc'] = spec['MolluscBOOL'].astype(int)
spec['C_Flowering Plant'] = spec['Fl_PlBOOL'].astype(int)

In [73]:
spec = spec.drop(columns = ['PlanarianBOOL', 'TardigradeBOOL', 'AnnelidBOOL', 'LiverwortBOOL', 'Pl_GymnoBOOL', 
                           'OtherBOOL', 'NDBOOL', 'ArachnidBOOL', 'Pl_CommBOOL', 'FernBOOL', 'ReptileBOOL',
                            'CrustaceanBOOL', 'NV_PLBOOL', 
                           'AmphibianBOOL', 'BirdBOOL', 'InsectBOOL',  'MammalBOOL', 
                           'FishBOOL', 'MolluscBOOL', 'Fl_PlBOOL'])

In [76]:
rankstat_ws = spec.groupby(['Watershed']).agg({'Huc_8': max, 'G5': sum,'G4': sum, 'G3': sum, 'G2': sum,'G1': sum, 
                                               'G12': sum, 'G23': sum, 'G34': sum, 'G45': sum, 
                                               'S5': sum,'S4': sum, 'S3': sum, 'S2': sum,'S1': sum, 'S12': sum, 
                                               'S23': sum, 'S34': sum,
                                               'LE': sum, 'LT': sum, 'summer': sum, 'T_Fungus': sum,
                                                'T_IntnlTer_EcoSystem': sum, 'T_An_Asslg': sum, 
                                               'T_IntnlVegClass': sum, 'T_NV_Plant': sum, 'T_INV_Animal': sum, 
                                               'T_VBRT_Animal': sum, 'T_VASC_Plant': sum, 'C_Planarian': sum, 
                                                'C_Tardigrade': sum, 'C_Annelid': sum, 'C_Liverwort': sum, 
                                                'C_Pl_Gymnosperm': sum, 'C_Other': sum, 'C_No_Data': sum, 
                                                'C_Arachnid': sum, 'C_Plt_Community': sum, 'C_Fern': sum,
                                                'C_Reptile': sum, 'C_Crustacean': sum, 'C_Non_Vasc_Plant': sum, 
                                                'C_Amphibian': sum, 'C_Bird': sum, 'C_Insect': sum, 'C_Mammal': sum, 
                                                'C_Fish': sum, 'C_Mollusc': sum, 'C_Flowering Plant': sum})

I want an idea of how many species don't have tags in these respective categories:

In [77]:
rankstat_ws['global_diff'] = rankstat_ws['summer'] - (rankstat_ws['G5'] + rankstat_ws['G4'] + 
                                                             rankstat_ws['G3'] + rankstat_ws['G2'] + 
                                                             rankstat_ws['G1'] + rankstat_ws['G12'] + 
                                                             rankstat_ws['G23'] + rankstat_ws['G34'] + 
                                                             rankstat_ws['G45'])

In [78]:
rankstat_ws['state_diff'] = rankstat_ws['summer'] - (rankstat_ws['S5'] + rankstat_ws['S4'] + 
                                                             rankstat_ws['S3'] + rankstat_ws['S2'] + 
                                                             rankstat_ws['S1'] + rankstat_ws['S12'] + 
                                                             rankstat_ws['S23'] + rankstat_ws['S34'])

In [79]:
rankstat_ws['fed_diff'] = rankstat_ws['summer'] - (rankstat_ws['LE'] + rankstat_ws['LT'])

In [80]:
rankstat_ws = rankstat_ws.reset_index(drop = False)

In [81]:
rankstat_ws.head()

,Watershed,Huc_8,G5,G4,G3,G2,G1,G12,G23,G34,...,C_Amphibian,C_Bird,C_Insect,C_Mammal,C_Fish,C_Mollusc,C_Flowering Plant,global_diff,state_diff,fed_diff
0,-,06010203,32,2,22,12,3,0,6,2,...,8,1,2,3,0,2,58,11,32,91
1,Barren River,05110002,1,19,9,0,0,2,1,1,...,0,1,1,3,34,1,1,0,3,47
2,Buffalo River,06040004,19,19,29,21,7,4,18,2,...,3,5,7,14,48,11,52,27,35,133
3,Caney Fork River,05130108,106,66,72,42,53,17,28,45,...,14,14,46,64,48,34,165,28,95,405
4,Cheatham Lake,05130202,19,13,27,3,5,4,2,10,...,7,5,1,2,5,4,58,5,19,82


In [82]:
rankstat_ws.describe()

,G5,G4,G3,G2,G1,G12,G23,G34,G45,S5,...,C_Amphibian,C_Bird,C_Insect,C_Mammal,C_Fish,C_Mollusc,C_Flowering Plant,global_diff,state_diff,fed_diff
count,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.00000,56.000000,56.0,...,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000,56.000000
mean,33.000000,21.767857,26.000000,9.678571,12.000000,3.642857,8.875000,11.62500,4.125000,0.0,...,6.750000,4.910714,5.053571,11.053571,19.732143,17.142857,63.857143,16.750000,34.267857,132.767857
std,30.397966,19.741043,21.909732,9.991100,13.509593,4.657071,9.656016,10.50119,3.963756,0.0,...,6.737278,4.753092,7.418453,11.920067,18.891548,18.558437,62.117609,16.871978,27.865581,108.515603
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,14.750000,6.750000,9.750000,2.000000,1.750000,0.000000,2.000000,2.75000,1.000000,0.0,...,1.750000,1.000000,0.750000,2.000000,5.000000,3.500000,17.000000,3.000000,13.500000,47.000000
50%,24.000000,17.500000,20.500000,7.000000,6.500000,2.500000,6.000000,8.50000,3.500000,0.0,...,4.000000,4.000000,2.500000,9.500000,14.500000,12.000000,42.000000,12.000000,25.500000,98.000000
75%,41.250000,27.250000,38.250000,16.250000,20.500000,5.250000,13.000000,17.50000,7.000000,0.0,...,11.750000,7.000000,7.000000,15.000000,30.500000,26.500000,101.250000,27.250000,51.250000,191.750000
max,134.000000,85.000000,81.000000,42.000000,53.000000,20.000000,45.000000,45.00000,15.000000,0.0,...,27.000000,23.000000,46.000000,64.000000,99.000000,73.000000,238.000000,68.000000,106.000000,424.000000


#### These ^ may be cool tables by ranks, types, etc.

In [83]:
rankstat_ws_geo = pd.merge(left = watersheds, right = rankstat_ws, left_on = 'HUC_8', right_on = 'Huc_8')

In [84]:
rankstat_ws_geo['id'] = rankstat_ws_geo.index.astype(str)

In [85]:
rankstat_ws_geo.head()

,REGION,SUBREGION,BASIN,SUBBASIN,HUC_2,HUC_4,HUC_6,HUC_8,ACRES,SQ_MILES,...,C_Bird,C_Insect,C_Mammal,C_Fish,C_Mollusc,C_Flowering Plant,global_diff,state_diff,fed_diff,id
0,Lower Mississippi Region,Lower Mississippi-Hatchie,Hatchie-Obion,Obion,08,0801,080102,08010202,841854.0,1315.0,...,9,0,7,14,4,27,5,22,75,0
1,Lower Mississippi Region,Lower Mississippi-Hatchie,Hatchie-Obion,South Fork Obion,08,0801,080102,08010203,740948.0,1158.0,...,5,0,4,15,0,12,0,11,47,1
2,Lower Mississippi Region,Lower Mississippi-Hatchie,Hatchie-Obion,North Fork Forked Deer,08,0801,080102,08010204,610687.0,954.0,...,7,0,1,2,0,6,1,8,19,2
3,Lower Mississippi Region,Lower Mississippi-Hatchie,Hatchie-Obion,South Fork Forked Deer,08,0801,080102,08010205,679073.0,1061.0,...,3,0,1,10,0,12,0,14,28,3
4,Lower Mississippi Region,Lower Mississippi-Hatchie,Hatchie-Obion,Forked Deer,08,0801,080102,08010206,45565.0,71.0,...,0,0,0,1,0,0,0,0,1,4


In [86]:
rankstat_ws_geo = rankstat_ws_geo[['id', 'Watershed', 'Huc_8', 'REGION', 'SUBREGION', 'BASIN', 'SUBBASIN', 
                                           'G5', 'G4', 'G3', 'G2', 'G1', 'G12', 'G23', 'G34', 'G45', 
                                           'S5', 'S4', 'S3', 'S2', 'S1', 'S12', 'S23', 'S34', 'LE', 'LT', 'summer', 
                                           'global_diff', 'state_diff', 'fed_diff', 'T_Fungus', 
                                           'T_IntnlTer_EcoSystem', 'T_An_Asslg', 'T_IntnlVegClass', 'T_NV_Plant', 
                                           'T_INV_Animal', 'T_VBRT_Animal', 'T_VASC_Plant', 'C_Planarian', 'C_Tardigrade', 
                                           'C_Annelid', 'C_Liverwort', 'C_Pl_Gymnosperm', 'C_Other', 'C_No_Data', 
                                           'C_Arachnid', 'C_Plt_Community', 'C_Fern', 'C_Reptile', 'C_Crustacean', 
                                           'C_Non_Vasc_Plant', 'C_Amphibian', 'C_Bird', 'C_Insect', 'C_Mammal', 'C_Fish', 
                                           'C_Mollusc', 'C_Flowering Plant', 'geometry']]

In [87]:
type(rankstat_ws_geo)

geopandas.geodataframe.GeoDataFrame

In [88]:
rankstat_ws_geo.to_file('../data/test/watershedbreakout.shp')